In [4]:
import string
import re
import codecs
import numpy as np
import pandas as pd
from sklearn import feature_extraction 
from sklearn import linear_model
from sklearn import pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [5]:
df = pd.read_csv('Language Detection.csv')

In [6]:
df.head()

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10337 entries, 0 to 10336
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      10337 non-null  object
 1   Language  10337 non-null  object
dtypes: object(2)
memory usage: 161.6+ KB


In [89]:
df.Language.unique()

array(['English', 'Malayalam', 'Hindi', 'Tamil', 'Portugeese', 'French',
       'Dutch', 'Spanish', 'Greek', 'Russian', 'Danish', 'Italian',
       'Turkish', 'Sweedish', 'Arabic', 'German', 'Kannada'], dtype=object)

In [7]:
x = df.Text
y = df.Language

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
encoded_y = le.fit_transform(y)

In [9]:
le.inverse_transform([3])

array(['English'], dtype=object)

In [10]:
data = []
for text in x:
    text = re.sub(r'[!@#$(),.n"%^*?:;~`0-9\n]', '', text)
    text = re.sub(r'[[]]', ' ', text)
    text = text.lower()
    data.append(text)

C:\Users\shankesh\AppData\Local\Temp\ipykernel_6900\2681050958.py:4: FutureWarning: Possible nested set at position 1
  text = re.sub(r'[[]]', ' ', text)


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x = cv.fit_transform(data).toarray()

In [12]:
xtrain, xtest, ytrain, ytest = train_test_split(x, encoded_y, train_size=0.8, random_state = 12)

In [13]:
xtrain.shape

(8269, 38700)

In [31]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(xtrain, ytrain)

MultinomialNB()

In [55]:
yhat = model.predict(xtest)

In [56]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(ytest, yhat)

In [57]:
accuracy

0.9714700193423598

In [54]:
def prediction(text):
    x = cv.transform([text]).toarray()
    lang = model.predict(x)
    lang = le.inverse_transform(lang)
    print('The Language is in ',lang[0])
    

In [51]:
def prediction2(text):
    x = cv.transform([text]).toarray()
    lang = model.predict(x)
    #lang = np.argmax(lang)
    lang = le.inverse_transform(lang)
    print('Neural The Language is in ',lang)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [24]:
model = Sequential()
model.add(Dense(500, input_dim=38700, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(17, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Train model
model.fit(xtrain, ytrain, epochs=4)

Epoch 1/4
259/259 [==============================] - 62s 226ms/step - loss: 0.4480 - accuracy: 0.8850
Epoch 2/4
259/259 [==============================] - 56s 215ms/step - loss: 0.0233 - accuracy: 0.9952
Epoch 3/4
259/259 [==============================] - 57s 222ms/step - loss: 0.0104 - accuracy: 0.9973
Epoch 4/4
259/259 [==============================] - 60s 233ms/step - loss: 0.0099 - accuracy: 0.9977


In [55]:
prediction2('أحتاج إلى شراء دراجة هوندا بعد حصولي على وظيفة')
prediction('أحتاج إلى شراء دراجة هوندا بعد حصولي على وظيفة')

Neural The Language is in  ['Arabic']
The Language is in  Arabic


In [56]:
prediction("это портал знаний на базе сообщества для профессионалов в области аналитики и данных.")
prediction2("это портал знаний на базе сообщества для профессионалов в области аналитики и данных.")

The Language is in  Russian
Neural The Language is in  ['Russian']


In [57]:
prediction("வணக்கம் ச மாப்ள")
prediction2("வணக்கம் ச மாப்ள")

The Language is in  Tamil
Neural The Language is in  ['Tamil']


In [58]:
prediction("എനിക്ക് ജോലി കിട്ടിയതിന് ശേഷം എനിക്ക് ഒരു ഹോണ്ട ബൈക്ക് വാങ്ങണം")
prediction2("എനിക്ക് ജോലി കിട്ടിയതിന് ശേഷം എനിക്ക് ഒരു ഹോണ്ട ബൈക്ക് വാങ്ങണം")

The Language is in  Malayalam
Neural The Language is in  ['Malayalam']


In [59]:
prediction("ನಾನು ಕೆಲಸ ಪಡೆದ ನಂತರ ನಾನು ಹೋಂಡಾ ಬೈಕ್ ಖರೀದಿಸಬೇಕಾಗಿದೆ")
prediction2("ನಾನು ಕೆಲಸ ಪಡೆದ ನಂತರ ನಾನು ಹೋಂಡಾ ಬೈಕ್ ಖರೀದಿಸಬೇಕಾಗಿದೆ")

The Language is in  Kannada
Neural The Language is in  ['Kannada']


In [60]:
prediction("ik moet een honda-fiets kopen nadat ik een baan heb gekregen")
prediction2("ik moet een honda-fiets kopen nadat ik een baan heb gekregen")

The Language is in  Dutch
Neural The Language is in  ['Dutch']


In [61]:
prediction("necesito comprar una bicicleta honda después de conseguir un trabajo")
prediction2("necesito comprar una bicicleta honda después de conseguir un trabajo")

The Language is in  Spanish
Neural The Language is in  ['Spanish']


In [62]:
prediction("Jag måste köpa en hondacykel efter att jag fått jobb")
prediction2("Jag måste köpa en hondacykel efter att jag fått jobb")

The Language is in  Sweedish
Neural The Language is in  ['Sweedish']


In [63]:
prediction("Ich muss mir ein Honda-Motorrad kaufen, nachdem ich einen Job bekommen habe")
prediction2("Ich muss mir ein Honda-Motorrad kaufen, nachdem ich einen Job bekommen habe")

The Language is in  German
Neural The Language is in  ['German']


In [64]:
prediction("Πρέπει να αγοράσω ένα ποδήλατο honda αφού βρω δουλειά")
prediction2("Πρέπει να αγοράσω ένα ποδήλατο honda αφού βρω δουλειά")

The Language is in  Greek
Neural The Language is in  ['Greek']


In [66]:
prediction("eu preciso comprar uma moto honda depois que eu conseguir um emprego")
prediction2("eu preciso comprar uma moto honda depois que eu conseguir um emprego")

The Language is in  Portugeese
Neural The Language is in  ['Portugeese']
